<a href="https://colab.research.google.com/github/albarabimakasa/internships/blob/main/Bara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Standard Data Science Libraries
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array

# Neural Net Preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Neural Net Layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

# Neural Net Training
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

from pickle import load


In [5]:
from google.colab import files
myfiles = files.upload()


Saving train.csv to train.csv


In [6]:
# Import the data
train_df = pd.read_csv('train.csv')


In [7]:
train_df

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [8]:
# Selecting Edgar Allen Poe as author style to emulate
author = train_df[train_df['author']=='EAP']["text"]
print('Number of training sentences: ',author.shape[0])

Number of training sentences:  7900


In [9]:

max_words = 50000 # Max size of the dictionary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(author.values)
sequences = tokenizer.texts_to_sequences(author.values)
print(sequences[:5])

[[19, 2397, 80, 1001, 29, 31, 177, 2, 4073, 1, 1960, 2, 11, 3024, 15, 7, 110, 157, 41, 2146, 3, 481, 4, 1, 149, 2147, 7, 393, 74, 114, 101, 439, 2, 1, 162, 32, 913, 6453, 136, 1, 380], [6, 21, 142, 150, 10, 5, 551, 2148, 319, 28, 16, 15, 20, 8999, 128, 1, 3025, 2398, 30, 171, 2, 1797, 697, 20, 180, 2148, 6454, 12, 33, 188, 2, 1, 869, 243, 522, 1264], [1, 6455, 203, 14, 19, 149, 180, 6456, 6, 1, 1357, 2, 1358, 9000, 3, 83, 2149, 10, 355, 140, 794], [1, 4074, 491, 6, 9001, 28, 11, 158], [7, 287, 9, 36, 48, 22, 73, 4, 644, 9002, 114, 101, 346, 4, 271, 2, 9003, 3, 81, 2, 1, 3026, 2, 6457, 3, 282, 53, 34, 6458, 19, 339, 22, 43, 97, 608, 7, 450, 4, 36, 133, 1191, 88, 12, 133, 71, 914, 1, 759, 3027, 2, 9, 1445, 1359, 18, 760, 12, 4973, 6, 1, 421, 9004, 9005, 7, 214, 9, 36, 48, 22, 3449, 3028, 98, 124, 1192, 4, 1, 92, 9006, 6, 3450, 3, 7, 761, 870, 9, 36, 55, 111, 32]]


In [10]:
# Flatten the list of lists resulting from the tokenization. This will reduce the list
# to one dimension, allowing us to apply the sliding window technique to predict the next word
text = [item for sublist in sequences for item in sublist]
vocab_size = len(tokenizer.word_index)

In [11]:
print('Vocabulary size in this corpus: ', vocab_size)

Vocabulary size in this corpus:  15713


In [12]:
# Training on 19 words to predict the 20th
sentence_len = 20
pred_len = 1
train_len = sentence_len - pred_len
seq = []
# Sliding window to generate train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])
# Reverse dictionary to decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Save tokenizer
# dump(tok, open('tokenizer.pkl', 'wb'))

In [13]:
# Each row in seq is a 20 word long window. We append he first 19 words as the input to predict the 20th word
trainX = []
trainy = []
for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

In [14]:
# define model
model = Sequential([
    Embedding(vocab_size+1, 50, input_length=train_len),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 50)            785700    
_________________________________________________________________
lstm (LSTM)                  (None, 19, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 15713)             1587013   
Total params: 2,523,613
Trainable params: 2,523,613
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.asarray(trainX),
         pd.get_dummies(np.asarray(trainy)),
         epochs = 100,
         batch_size = 10240,
         #callbacks = callbacks_list(),
         verbose = 2)


Epoch 1/100
20/20 - 17s - loss: 9.3984 - accuracy: 0.0056
Epoch 2/100
20/20 - 9s - loss: 7.1926 - accuracy: 0.0401
Epoch 3/100
20/20 - 9s - loss: 6.8398 - accuracy: 0.0744
Epoch 4/100
20/20 - 9s - loss: 6.8074 - accuracy: 0.0744
Epoch 5/100
20/20 - 9s - loss: 6.8010 - accuracy: 0.0744
Epoch 6/100
20/20 - 9s - loss: 6.7992 - accuracy: 0.0744
Epoch 7/100
20/20 - 9s - loss: 6.7984 - accuracy: 0.0744
Epoch 8/100
20/20 - 9s - loss: 6.7976 - accuracy: 0.0744
Epoch 9/100
20/20 - 9s - loss: 6.7972 - accuracy: 0.0744
Epoch 10/100
20/20 - 9s - loss: 6.7968 - accuracy: 0.0744
Epoch 11/100
20/20 - 9s - loss: 6.7959 - accuracy: 0.0744
Epoch 12/100
20/20 - 9s - loss: 6.7953 - accuracy: 0.0744
Epoch 13/100
20/20 - 9s - loss: 6.7948 - accuracy: 0.0744
Epoch 14/100
20/20 - 9s - loss: 6.7933 - accuracy: 0.0744
Epoch 15/100
20/20 - 9s - loss: 6.7916 - accuracy: 0.0744
Epoch 16/100
20/20 - 9s - loss: 6.7875 - accuracy: 0.0744
Epoch 17/100
20/20 - 9s - loss: 6.7765 - accuracy: 0.0744
Epoch 18/100
20/20 - 9

In [18]:
# fit model
model.fit(np.asarray(trainX), pd.get_dummies(np.asarray(trainy)), batch_size=128, epochs=100)

Epoch 1/100
1571/1571 [==============================] - 42s 27ms/step - loss: 5.6572 - accuracy: 0.1438
Epoch 2/100
1571/1571 [==============================] - 42s 27ms/step - loss: 5.3876 - accuracy: 0.1534
Epoch 3/100
1571/1571 [==============================] - 42s 27ms/step - loss: 5.2467 - accuracy: 0.1583
Epoch 4/100
1571/1571 [==============================] - 42s 27ms/step - loss: 5.1154 - accuracy: 0.1627
Epoch 5/100
1571/1571 [==============================] - 41s 26ms/step - loss: 4.9842 - accuracy: 0.1677
Epoch 6/100
1571/1571 [==============================] - 41s 26ms/step - loss: 4.8589 - accuracy: 0.1726
Epoch 7/100
1571/1571 [==============================] - 41s 26ms/step - loss: 4.7435 - accuracy: 0.1776
Epoch 8/100
1571/1571 [==============================] - 41s 26ms/step - loss: 4.6356 - accuracy: 0.1832
Epoch 9/100
1571/1571 [==============================] - 41s 26ms/step - loss: 4.5390 - accuracy: 0.1893
Epoch 10/100
1571/1571 [==============================]

In [19]:
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
model.save('model_weights.hdf5')

In [20]:
def gen(model,seq,max_len = 20):
    ''' Generates a sequence given a string seq using specified model until the total sequence length
    reaches max_len'''
    # Tokenize the input string
    tokenized_sent = tokenizer.texts_to_sequences([seq])
    max_len = max_len+len(tokenized_sent[0])
    # If sentence is not as long as the desired sentence length, we need to 'pad sequence' so that
    # the array input shape is correct going into our LSTM. the `pad_sequences` function adds 
    # zeroes to the left side of our sequence until it becomes 19 long, the number of input features.
    while len(tokenized_sent[0]) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-19:],maxlen=19)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax()+1)
        
    return " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))

In [23]:
model_list = [model]

In [24]:

def test_models(test_string,sequence_length= 50, model_list = model_list):
    '''Generates output given input test_string up to sequence_length'''
    print('Input String: ', test_string)
    for counter,model in enumerate(model_list):
        print("Model ", counter+1, ":")
        print(gen(model,test_string,sequence_length))
    pass

In [25]:

test_models('This process however afforded me', 10)

Input String:  This process however afforded me
Model  1 :
this process however afforded me in the smallest i shall say that the thief is


In [26]:
test_models(author.iloc[3709])

Input String:  what avails the vigilance against the Destiny of man?
Model  1 :
what avails the vigilance against the destiny of man inclined to the angels nor bear about buried to have something to excite the lee or so moreover not the division been west but the natives suspend it by a cord seldom have not a lamp and then i was enabled to confess the women about seen the beetle here


In [27]:

test_models(author.iloc[58],50)

Input String:  By these means for they were ignorant men I found little difficulty in gaining them over to my purpose.
Model  1 :
by these means for they were ignorant men i found little difficulty in gaining them over to my purpose with much suffering afterward in connexion with my medical windows i interposed the frenchman with which the deity expressed even he was the only ones of the same large and thoughtful frequented in the title of the bishop of chichester stay for me to be johannisberger now the medium week


In [28]:
test_models(author.iloc[70])

Input String:  In the meantime it was folly to grieve, or to think.
Model  1 :
in the meantime it was folly to grieve or to think of the intruder in the ordinary character in the world to be sure was generally is was making to revolve at slack and as i put in the street place or re metzengerstein i am not seven employed during the midway who were so well express that the prefect knows


In [29]:
test_models(author.iloc[7800])

Input String:  First of all I dismembered the corpse.
Model  1 :
first of all i dismembered the corpse in the money itself from the library well had never visited or knocked firmly away up de peg throw the parchment a charitable and had been supposed to me the original object of its moral and and social spots sustain energy if i had been preferred that as i confided


In [30]:
test_models(author.iloc[7120])

Input String:  Here, however, are Moissart, Voissart, Croissart, and Froissart, all in the direct line of descent.
Model  1 :
here however are moissart voissart croissart and froissart all in the direct line of descent the whole story that is law the whole philosophy knows his birds always dangerous extent i turned the spectacles and so collected its escape at length and dismissing me with similes it in the essence the task of years or of many weeks or fish a wild and quaint ground


In [31]:
test_models(author.iloc[5121])

Input String:  "There are two windows in the chamber.
Model  1 :
there are two windows in the chamber which oppressed me than the beast the nerve conveys luminous similar and harsh and money and puppy melancholy the articles had been deviates from the wings of nature by the wailings and in the silk of the bizarre overshadowed the greatest from the earth's familiar tones of the atmosphere is
